# EG 24/25 - TPC3 (Lista - Lark)

## Maria Cunha e Tomás Campinho - Gr10

Link Google Colab: https://colab.research.google.com/drive/1IYqNLf13kmUP0zxJWsbUdbKInlW5g40S?usp=sharing

Desenvolva uma GIC para definir uma linguagem que permita escrever listas mistas de números e palavras, de tal forma que as 3 frases abaixo sejam frases válidas dessa linguagem:

LISTA 1 .

Lista aaa .

Lista 1, 2, agora, 3, 4, fim, 7, 8.

Resolva as seguintes alíneas recorrendo ao uso de Transformer :

(a) Calcule, retorne e imprima quantos elementos existem numa lista.

(b) Identifique, retorne e imprima o elemento que mais se repete numa lista.

(c) Calcule, retorne e imprima a soma de todos os números que se encontrem entre as palavras agora e fim de uma lista.

(d) Verifique se uma lista é válida de acordo com os seguintes parâmetros :

Caso a palavra "agora" ocorra então a palavra "fim" também tem de aparecer posteriormente na lista e o contrário também.
Entre "agora" e "fim" tem de aparecer pelo menos um número.

In [22]:
!pip install lark

In [36]:
from lark import Lark, Transformer, Token, Tree

grammar = r'''
start: lista
lista: elems DOT
elems: elem ("," elem)*
elem: num
    | pal
    | AGORA
    | FIM

num: /\d+(\.\d+)?/
pal: /[a-zA-Z]+/
AGORA: "agora"
FIM: "fim"
DOT: "."

%import common.WS
%ignore WS
'''

parser = Lark(grammar, start='start', parser='lalr')

from lark import Transformer, Tree, Token

class CalcTransformer(Transformer):
    def __init__(self):
        self.count = 0
        self.elements = []
        self.soma = 0
        self.agora_stack = []

    def lista(self, items):
        return items[0]

    def elems(self, items):
        filtered_items = [item for item in items if item not in {",", "."}]

        # ao contar elementos, não contar as palavras agora e fim
        self.elements = [item for item in filtered_items if item not in {"agora", "fim"}]
        self.count = len(self.elements)

        # fazer o tracking da soma entre "agora" and "fim" com uma stack
        self.soma = 0
        self.agora_stack = []

        for item in filtered_items:
            if item == "agora":
                self.agora_stack.append(True)
            elif item == "fim":
                if self.agora_stack:
                    self.agora_stack.pop()
            elif self.agora_stack and isinstance(item, (int, float)):
                self.soma += item  # soma apenas um "agora" estiver aberto
            elif self.agora_stack and isinstance(item, str) and item.replace('.', '', 1).isdigit():
                self.soma += float(item)

        return filtered_items

    def elem(self, item):
        if isinstance(item[0], Tree):
            return item[0].children[0].value  # num
        elif isinstance(item[0], Token):
            return item[0].value  # pal, agora, fim
        return None

    def most_common(self):
        if not self.elements:
            return None
        return max(set(self.elements), key=self.elements.count, default=None)


text = "1, agora, 2, agora, 3, fim, 4, fim, 5, 6, 6, agora, 10, fim ."

tree = parser.parse(text)
calctransformer = CalcTransformer()
calctransformer.transform(tree)

print("Numero de elementos na lista:", calctransformer.count)
print("Elemento mais comum:", calctransformer.most_common())
print("Soma entre agora e fim:", calctransformer.soma)

Numero de elementos na lista: 8
Elemento mais comum: 6
Soma entre agora e fim: 19.0
